<a href="https://colab.research.google.com/github/lizaoh/smp_program_data/blob/main/smp2009_extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top of Script

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymupdf
!pip install pymupdf-layout
!pip install pymupdf4llm
!pip install rapidfuzz
import glob
import os
import pathlib
import pymupdf
import pymupdf.layout
import pymupdf4llm
import re
import pandas as pd
import unicodedata
from rapidfuzz import process, fuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: PyMuPDF
    Found existing installation: PyMuPDF 1.26.7
    Uninstalling PyMuPDF-1.26.7:
      Successfully uninstalled PyMuPDF-1.26.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.0 MB/s eta 0:00:00


In [3]:
pdfs_path = '/content/drive/MyDrive/math_psych_work/Conference Programs/'

# Functions
Created with help from GPT 5.2, but some are my own code just turned into a function.

In [80]:
def clean_text(text):
    if not text:
        return text

    text = re.sub(r' \n\n\d{1,3} \n\n', ' ', text)
    text = re.sub(r'\s*\n\s*', ' ', text)    # replace newlines with spaces
    text = re.sub(r'-\s+', '', text)         # get rid of "- "; will fix actual
                                             # hyphenated words manually
    text = text.strip()
    text = fix_ligatures(text)

    return text

In [81]:
LIGATURE_MAP = {
    "ﬁ": "fi",
    "ﬂ": "fl",
    "ﬃ": "ffi",
    "ﬄ": "ffl",
    "ﬀ": "ff",
    "ﬅ": "ft",
    "ﬆ": "st",
    "Æ": 'ffi'
}

def fix_ligatures(text):
    # Replace known ligatures
    for bad, good in LIGATURE_MAP.items():
        text = text.replace(bad, good)

    # Replace any private-use ligature (common in PDFs)
    cleaned_chars = []
    for ch in text:
        name = unicodedata.name(ch, "")
        if "LIGATURE" in name.upper():
            # Try to break it apart: remove spaces and lowercase
            base = name.split("LIGATURE")[-1]
            base = base.replace(" ", "").lower()
            cleaned_chars.append(base)
        else:
            cleaned_chars.append(ch)

    return "".join(cleaned_chars)

# Program

Program pdf is table of 141 entries with "[author last name]: [title]". A couple of entries I didn't count are discussions and one where one paper has 2 time slots (Shiffrin on model selection).

Abstract pdf has 176 (includes posters) entries with some location and date info, then title in bold, period, authors and affiliations (interdispersed depending on if share institutions), period, then abstract. But most times, affiliation has space after it (before the following comma) but author doesn't.

## Grab text from the pdf

In [82]:
year = '2009'
abstracts = pymupdf.open(pdfs_path + f'smp{year}_abstracts.pdf')

In [83]:
abstract_text = pymupdf4llm.to_text(abstracts)

=== Document parser messages ===
Full-page OCR on page.number=0/1.
Full-page OCR on page.number=0/1.



In [84]:
abstract_entries = abstract_text.split("\n\nAbstracts For Talks \n\n(1) ", 1)[1]
talk_entries, poster_entries = abstract_entries.split(" \n\n73 \n\n", 1)
poster_entries = poster_entries.split(" \n\n92 \n\n", 1)[0]

In [85]:
talk_entries[:1000]

'\n\nAug 02, 09:00–09:20 \n\nLecture Hall A Robust versus Optimal Strategies for Two-Alternative Forced-Choice Tasks. Philip Holmes, Princeton University , Miriam Zacksenhouse, Technion - Israel Institute of Technology , Rafal Bogacz, University of Bristol . The driftdiffusion model predicts a speed-accuracy tradeoff that maximizes reward rate in twoalternative forced-choice tasks. This can be expressed as an optimal performance curve that relates normalized decision times to error rates under varying task conditions. Human behavioral data indicate that 30% of subjects achieve optimality, and in this talk we propose that, in allowing for uncertainties, subjects might exercise robust instead of optimal strategies. We describe two such strategies: maximin and robust-satisficing. The former supposes maximization of guaranteed performance under a presumed level of uncertainty; the latter assumes a required performance level and maximizes the uncertainty under which it can be assured. We sh

In [86]:
poster_entries[:1000]

'Abstracts For Posters \n\n(P1) \n\nAug 03, 18:00–19:30 \n\nPoster session Strong Knowledge in Psychology and Artificial Intelligence. Carlos Pelta, Complutense University of Madrid . In this contribution it is introduced a formal system (Ks) expressing the notion of strong knowledge. An agent has strong knowledge of that expressed by the formula A if and only if all the possible worlds in which A is applicable for the agent, can be finitely determined by the same. We find applications in the Psychology of planning and in AI (simple movement automatons). For instance, in classical tasks of planning like (TOL), the participants manage only a small subset of possible states whilst solving those puzzles. They are situations of closed knowledge imposed by the restrictions of the proper task. They can be analysed using our formal system. \n\n==> picture [24 x 13] <==\n\nAug 03, 18:00–19:30 \n\nPoster session \n\nAIC and BIC in Model Selection and Inference from Psychological Data. Caterina 

## Clean up entries

In [87]:
# Get individual entries (first part is just junk)
indiv_talk_entries = re.split(r"Hall [A-C] ", talk_entries)[1:]
indiv_poster_entries = re.split(r"Poster session ", poster_entries)[1:]
indiv_entries = indiv_talk_entries + indiv_poster_entries

In [88]:
indiv_entries[0]

'Robust versus Optimal Strategies for Two-Alternative Forced-Choice Tasks. Philip Holmes, Princeton University , Miriam Zacksenhouse, Technion - Israel Institute of Technology , Rafal Bogacz, University of Bristol . The driftdiffusion model predicts a speed-accuracy tradeoff that maximizes reward rate in twoalternative forced-choice tasks. This can be expressed as an optimal performance curve that relates normalized decision times to error rates under varying task conditions. Human behavioral data indicate that 30% of subjects achieve optimality, and in this talk we propose that, in allowing for uncertainties, subjects might exercise robust instead of optimal strategies. We describe two such strategies: maximin and robust-satisficing. The former supposes maximization of guaranteed performance under a presumed level of uncertainty; the latter assumes a required performance level and maximizes the uncertainty under which it can be assured. We show that maximin performance curves for unce

In [89]:
parsed_entries = []

for entry in indiv_entries:
  auths = []
  affs = []

  no_trailing_junk = entry.split(".")[:-1]
  cleaned_entry = clean_text(".".join(no_trailing_junk))

  # Get title
  title, rest_of_entry = re.split(r"[.?]", cleaned_entry, 1)
  if re.search(r"\s*\d{2}\s*", title):
    cleaned_title = re.sub(r"\s*\d{2}\s*", "", title).strip()
  else:
    cleaned_title = title.strip()

  # Get authors and affiliations
  if ' . ' in rest_of_entry:
    auth_and_aff, abstract = rest_of_entry.split(' . ', 1)

    # Affiliations in most cases have a space between it and the comma
    # but authors have no space (sometimes does but I'll fix manually)
    split_auth_aff = auth_and_aff.split(', ')
    for auth_aff in split_auth_aff[:-1]:
      if auth_aff[-1] == ' ':
        affs.append(auth_aff.strip())
      else:
        auths.append(auth_aff.strip())
    affs.append(split_auth_aff[-1].strip()) # last part is always an affiliation
  else:
    abstract = rest_of_entry

  parsed_entries.append({
      'year': year,
      'author(s)': ', '.join(auths),
      'affiliation(s)': '; '.join(affs),
      'title': cleaned_title,
      'type': '',
      'abstract': abstract + '.'  # adding back last period
  })

In [90]:
parsed_entries[:2]

[{'year': '2009',
  'author(s)': 'Philip Holmes, Miriam Zacksenhouse, Rafal Bogacz',
  'affiliation(s)': 'Princeton University; Technion Israel Institute of Technology; University of Bristol',
  'title': 'Robust versus Optimal Strategies for Two-Alternative Forced-Choice Tasks',
  'type': '',
  'abstract': 'The driftdiffusion model predicts a speed-accuracy tradeoff that maximizes reward rate in twoalternative forced-choice tasks. This can be expressed as an optimal performance curve that relates normalized decision times to error rates under varying task conditions. Human behavioral data indicate that 30% of subjects achieve optimality, and in this talk we propose that, in allowing for uncertainties, subjects might exercise robust instead of optimal strategies. We describe two such strategies: maximin and robust-satisficing. The former supposes maximization of guaranteed performance under a presumed level of uncertainty; the latter assumes a required performance level and maximizes th

# Create df and convert to csv

In [91]:
# Skip first entry because extraneous text
df = pd.DataFrame(parsed_entries, columns=["year", "author(s)", "affiliation(s)", "title", "type", "abstract"])

In [92]:
df.head()

,year,author(s),affiliation(s),title,type,abstract
0,2009,"Philip Holmes, Miriam Zacksenhouse, Rafal Bogacz",Princeton University; Technion Israel Institut...,Robust versus Optimal Strategies for Two-Alter...,,The driftdiffusion model predicts a speed-accu...
1,2009,"Marius Usher, Konstantinos Tsetsos, Andrei Teo...",Tel-Aviv University; University College London...,Contrasting Models of Perceptual Choice,,A number of neurocomputational models have bee...
2,2009,Jochen Ditterich,University of California,Perceptual Decisions between Multiple Alternat...,,The 2AFC version of the random dot motion dire...
3,2009,"Rafal Bogacz, Kevin Gurney",University of Bristol; University of Sheffield,Optimal Decision Making in the Cortico-Basal-G...,,It is well established that the patterns of re...
4,2009,Michael Frank,Brown University,Interactive Dynamics between Frontal Cortex an...,,The frontal cortex and basal ganglia (BG) inte...


In [93]:
df.to_csv(f"/content/drive/MyDrive/math_psych_work/csv/smp{year}_program.csv", index=False)